## Transfer raw data (cdr flows) to graph format

## This is a data pre-processing for centrality measurements.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

import timeit

import matplotlib.pyplot as plt
%matplotlib notebook

# 1. Read Data

In [4]:
home = "/Users/yanchao/000_2020MIT_FALL/SL/data/towerOD/"
p1 = 'feb.csv'

p = pd.read_csv(home + p1, low_memory = False)
p = p[['des_id', 'count', 'ori_id', 'Latitude_ori','longitude_ori', 'Latitude_des', 'longitude_des',]]
p

,des_id,count,ori_id,Latitude_ori,longitude_ori,Latitude_des,longitude_des
0,1001,2.250000,1000,8.480993,-13.218795,8.483741,-13.256992
1,1001,3.000000,1000,8.480993,-13.218795,8.483741,-13.256992
2,1001,1.000000,1000,8.480993,-13.218795,8.483741,-13.256992
3,1001,1.333333,1000,8.480993,-13.218795,8.483741,-13.256992
4,1001,1.666667,1000,8.480993,-13.218795,8.483741,-13.256992
...,...,...,...,...,...,...,...
646407,2255,1.000000,4081,7.858860,-11.187200,7.457970,-11.238130
646408,2255,1.000000,4084,7.869370,-11.192250,7.457970,-11.238130
646409,2255,1.000000,4089,7.883790,-11.189700,7.457970,-11.238130
646410,2255,1.000000,4091,7.869370,-11.192250,7.457970,-11.238130


# 2. Pre-processing

## 2.1. Add real locations to each tower (node)

In [5]:
def generate_loc(dataframe, lat_name, lon_name, loc_name):
    """
    INPUT:
    dataframe we use to generate a location. e.g., tower_trips
    lat_name, column name for latitude
    lon_name, column name for longitude
    
    OUTPUT:
    dataframe with a new added column called 'loc', is a tuple containing (lat, lon).
    """
    
    location = []
    dataframe[lat_name] = round(dataframe[lat_name],8) # to avoid some comupter store issue. For example, -13.218795 may be stored as -13.218795000000002
    dataframe[lon_name] = round(dataframe[lon_name],8)
    for label, row in dataframe.iterrows():
        t = (row[lat_name],row[lon_name])
        location.append(t) 
    dataframe[loc_name] = location
    return dataframe

In [8]:
p = generate_loc(p,'Latitude_ori' , 'longitude_ori', 'oriloc')
p = generate_loc(p,'Latitude_des', 'longitude_des', 'desloc')
p

,des_id,count,ori_id,Latitude_ori,longitude_ori,Latitude_des,longitude_des,oriloc,desloc
0,1001,2.250000,1000,8.480993,-13.218795,8.483741,-13.256992,"(8.480993, -13.218795)","(8.483741, -13.256992)"
1,1001,3.000000,1000,8.480993,-13.218795,8.483741,-13.256992,"(8.480993, -13.218795)","(8.483741, -13.256992)"
2,1001,1.000000,1000,8.480993,-13.218795,8.483741,-13.256992,"(8.480993, -13.218795)","(8.483741, -13.256992)"
3,1001,1.333333,1000,8.480993,-13.218795,8.483741,-13.256992,"(8.480993, -13.218795)","(8.483741, -13.256992)"
4,1001,1.666667,1000,8.480993,-13.218795,8.483741,-13.256992,"(8.480993, -13.218795)","(8.483741, -13.256992)"
...,...,...,...,...,...,...,...,...,...
646407,2255,1.000000,4081,7.858860,-11.187200,7.457970,-11.238130,"(7.85886, -11.1872)","(7.45797, -11.23813)"
646408,2255,1.000000,4084,7.869370,-11.192250,7.457970,-11.238130,"(7.86937, -11.19225)","(7.45797, -11.23813)"
646409,2255,1.000000,4089,7.883790,-11.189700,7.457970,-11.238130,"(7.88379, -11.1897)","(7.45797, -11.23813)"
646410,2255,1.000000,4091,7.869370,-11.192250,7.457970,-11.238130,"(7.86937, -11.19225)","(7.45797, -11.23813)"


## 2.2 Aggregate based on Unique Locations

In [13]:
# count the numbers of origin-destination pairs
p1 = p.groupby(['oriloc','desloc'])['count'].sum().reset_index()
p1

,oriloc,desloc,count
0,"(6.96844, -11.57535)","(7.02457, -11.38846)",153.300000
1,"(6.96844, -11.57535)","(7.15579, -11.37431)",27.500000
2,"(6.96844, -11.57535)","(7.315155, -11.306095)",12.500000
3,"(6.96844, -11.57535)","(7.31859, -11.84826)",16.500000
4,"(6.96844, -11.57535)","(7.350478, -11.717965)",28.266667
...,...,...,...
50210,"(9.916538, -11.449949)","(9.661688, -11.718794)",1.000000
50211,"(9.916538, -11.449949)","(9.688573, -11.720624)",1.000000
50212,"(9.916538, -11.449949)","(9.709185, -11.548924)",56.750000
50213,"(9.916538, -11.449949)","(9.854077, -11.322257)",75.100000


In [19]:
# filtered out those tower location of records more than 30 per day.
p2 = p1[p1['count']>30]
p2

,oriloc,desloc,count
0,"(6.96844, -11.57535)","(7.02457, -11.38846)",153.300000
31,"(7.02457, -11.38846)","(6.96844, -11.57535)",126.950000
32,"(7.02457, -11.38846)","(7.15579, -11.37431)",125.500000
33,"(7.02457, -11.38846)","(7.315155, -11.306095)",104.650000
61,"(7.02457, -11.38846)","(7.940896, -11.726057)",33.483333
...,...,...,...
50153,"(9.86506, -11.42903)","(9.916538, -11.449949)",449.750000
50209,"(9.916538, -11.449949)","(9.589409, -11.546852)",64.500000
50212,"(9.916538, -11.449949)","(9.709185, -11.548924)",56.750000
50213,"(9.916538, -11.449949)","(9.854077, -11.322257)",75.100000


In [20]:
# test if the filtered data (towers) have locations that match with the real location pairs
node_list = list(p2.desloc.unique())
new = list(p2.oriloc.unique())
for ele in new:
    if ele not in node_list:
        node_list.append(ele)


In [21]:
len(node_list) # result indicates the match 

319

# 3. Generate Origin-destination Pairs

## 3.1 Location Pairs

In [22]:
df = pd.DataFrame()
node = []
pos = []
for ii in range(len(node_list)):
    node.append(str(ii))
    pos.append(node_list[ii])
    
df['node'] = node
df['pos'] = pos
df

,node,pos
0,0,"(7.02457, -11.38846)"
1,1,"(6.96844, -11.57535)"
2,2,"(7.15579, -11.37431)"
3,3,"(7.315155, -11.306095)"
4,4,"(7.940896, -11.726057)"
...,...,...
314,314,"(9.688573, -11.720624)"
315,315,"(9.661688, -11.718794)"
316,316,"(9.854077, -11.322257)"
317,317,"(9.86506, -11.42903)"


In [ ]:
df.to_csv(home + 'loc.csv')

## 3.2. Generate location pairs for origin towers and destination towers

In [28]:
p2 = p2.merge(df, left_on = 'oriloc', right_on = 'pos', how = 'inner')
p2 = p2.rename(columns = {'node':'orinode'})
p2 = p2.merge(df, left_on = 'desloc', right_on = 'pos', how = 'inner')
p2 = p2.rename(columns = {'node':'desnode'})

p2 = p2[['orinode','desnode','count','oriloc','desloc']]
p2

,orinode,desnode,count,oriloc,desloc
0,1,0,153.300000,"(6.96844, -11.57535)","(7.02457, -11.38846)"
1,2,0,115.950000,"(7.15579, -11.37431)","(7.02457, -11.38846)"
2,3,0,109.700000,"(7.315155, -11.306095)","(7.02457, -11.38846)"
3,36,0,31.583333,"(7.863969, -11.196261)","(7.02457, -11.38846)"
4,5,0,120.650000,"(7.96143, -11.7417)","(7.02457, -11.38846)"
...,...,...,...,...,...
12846,318,317,428.500000,"(9.916538, -11.449949)","(9.86506, -11.42903)"
12847,256,318,63.300000,"(9.589409, -11.546852)","(9.916538, -11.449949)"
12848,302,318,60.250000,"(9.709185, -11.548924)","(9.916538, -11.449949)"
12849,316,318,76.550000,"(9.854077, -11.322257)","(9.916538, -11.449949)"


In [29]:
p2.to_csv(home + 'feb_graph.csv')